In [4]:
# read file to inspect it

with open('combined.txt', 'r', encoding='utf-') as f:
    text = f.read()

In [6]:
print("length of dataset in chars: ", len(text))

length of dataset in chars:  851155


In [7]:
print(text[:1000])

Boa constriktor
Іван Франко
ПОВІСТЬ
Герман Гольдкремер встав нині дуже злий. Він все такий, кілько разів йому лучається ночувати в Бориславі. А лучається то щотижня раз, в п'ятницю, коли приїздить сюди з Дрогобича оглянути роботу і виплатити ріпникам. Герман Гольдкремер, хоть маєток його доходить до мільйона, ніколи не звірюе чужим очам надзору ані чужим рукам виплати. У нього в Дрогобичі своя камениця, порядна, нова, ясна, — сказати не жаль. А тут приходиться йому ночувати в дерев'янім домику, серед магазинів, завалених бочками кип'ячки та величезними грудами воску. Правда, той домик, поставлений його коштом, все-таки найпорядніший і найкраще положений на весь Борислав, — але знов дарма річ рівняти його до дрогобицьких будинків. Хоть стіни білі і вікна ясні, та вид навкруги сумний, понурий, поганий: купи хворосту, купи глини, брудні магазини та ще брудніші помешкання людські. Ні зелені свіжої, ні виду всміхненого не побачиш. Воздух удушливий, загуслий від нафтового сопуху; у Германа в

In [8]:
# all the unique chars that occur in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#'()*,-./0123456789:;=?ABCEFGHIJLMNOPRSTVWXZ[]^_abcdefghijklmnopqrstuvwxyz}áäêüćęłňřśšż́ЄІЇАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёєії–—‘’“”•…№
171


In [11]:
# create a mapping from chars to integers; naive tokenizer implementation
stoi = {ch: i for i,ch in enumerate(chars)}
itos = {i: ch for i,ch in enumerate(chars)} 

encode = lambda s: [stoi[c] for c in s] # encoder: convert string to list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: convert list of integers back to string

print(encode("дайте ми кебаба"))
print(decode(encode("ти є файна жаба")))

[130, 126, 135, 144, 131, 1, 138, 134, 1, 136, 131, 127, 126, 127, 126]
ти є файна жаба


In [ ]:
!pip3 install torch

In [16]:
# encoding the entire text dataset and store in into a torch tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([851155]) torch.int64
tensor([ 28,  66,  52,   1,  54,  66,  65,  70,  71,  69,  60,  62,  71,  66,
         69,   0,  93, 128, 126, 139,   1, 115, 142, 126, 139, 136, 140,   0,
        110, 109,  97,  93, 112, 113, 122,   0,  98, 131, 142, 138, 126, 139,
          1,  98, 140, 137, 154, 130, 136, 142, 131, 138, 131, 142,   1, 128,
        143, 144, 126, 128,   1, 139, 134, 139, 160,   1, 130, 145, 132, 131,
          1, 133, 137, 134, 135,  11,   1,  97, 160, 139,   1, 128, 143, 131,
          1, 144, 126, 136, 134, 135,   9,   1, 136, 160, 137, 154, 136, 140,
          1, 142, 126, 133, 160, 128,   1, 135, 140, 138, 145,   1, 137, 145,
        149, 126, 159, 144, 154, 143, 157,   1, 139, 140, 149, 145, 128, 126,
        144, 134,   1, 128,   1,  96, 140, 142, 134, 143, 137, 126, 128, 160,
         11,   1,  95,   1, 137, 145, 149, 126, 159, 144, 154, 143, 157,   1,
        144, 140,   1, 151, 140, 144, 134, 132, 139, 157,   1, 142, 126, 133,
          9,   1, 128,   1, 141

In [18]:
# split up the data into train and validation sets

n = int(0.9*len(data)) # first 90% is train data, rest is val
train_data = data[:n]
val_data = data[n:]

In [19]:
block_size = 8
train_data[:block_size+1]

tensor([28, 66, 52,  1, 54, 66, 65, 70, 71])

In [20]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

when input is tensor([28]) the target is: 66
when input is tensor([28, 66]) the target is: 52
when input is tensor([28, 66, 52]) the target is: 1
when input is tensor([28, 66, 52,  1]) the target is: 54
when input is tensor([28, 66, 52,  1, 54]) the target is: 66
when input is tensor([28, 66, 52,  1, 54, 66]) the target is: 65
when input is tensor([28, 66, 52,  1, 54, 66, 65]) the target is: 70
when input is tensor([28, 66, 52,  1, 54, 66, 65, 70]) the target is: 71


In [21]:
torch.manual_seed(1999)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the max context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[160, 130, 141, 131, 136, 145, 128, 126],
        [126, 143, 144, 159,   9,   1, 151, 126],
        [  1, 128, 134, 143, 144, 126, 142, 149],
        [126, 159, 144, 154, 143, 157,   1, 130]])
targets:
torch.Size([4, 8])
tensor([[130, 141, 131, 136, 145, 128, 126, 128],
        [143, 144, 159,   9,   1, 151, 126, 143],
        [128, 134, 143, 144, 126, 142, 149, 126],
        [159, 144, 154, 143, 157,   1, 130, 134]])
---
when input is [160] the target: 130
when input is [160, 130] the target: 141
when input is [160, 130, 141] the target: 131
when input is [160, 130, 141, 131] the target: 136
when input is [160, 130, 141, 131, 136] the target: 145
when input is [160, 130, 141, 131, 136, 145] the target: 128
when input is [160, 130, 141, 131, 136, 145, 128] the target: 126
when input is [160, 130, 141, 131, 136, 145, 128, 126] the target: 128
when input is [126] the target: 143
when input is [126, 143] the target: 144
when input is [126, 143, 144] the 

In [22]:
print(xb)

tensor([[160, 130, 141, 131, 136, 145, 128, 126],
        [126, 143, 144, 159,   9,   1, 151, 126],
        [  1, 128, 134, 143, 144, 126, 142, 149],
        [126, 159, 144, 154, 143, 157,   1, 130]])


In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1999)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range (max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes B, C
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


torch.Size([32, 171])
tensor(5.6773, grad_fn=<NllLossBackward0>)

mФз=ш67шbьAЄюЗPr}єУБüFšч-
LкГЕ5ЇШnl=лZБ-ЩЭмЯХš3ь‘їМiіCSAOtВЭр(М(ЦśnłА9FЖЖNh/áü[áМмш’!_П…́aäЩРAJr=лn^


In [28]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [34]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4950947761535645


In [43]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype = torch.long), max_new_tokens=300)[0].tolist()))


УCsż[ютося.
— пріня, іда кум ве гобумупі влю хосвстрогоб таżаро?
Якобейоленькувкогродниливіви тихо То-чаву мимій спим ї шHПо сту, гоно ї кокупалови, тиїї дагоку в кули: стомор заве.
Додо маго муле нобу яюаро ві ріся йобмиціша ск кідо зисті. сіле7l/послюви до ся? киміся ін по впрившімух Прідцероя.
V



# The mathemtical trick in self-attention

In [44]:
# toy example:

torch.manual_seed(1999)
B,T,C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [63]:
# we want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [47]:
x[0]

tensor([[ 0.2811,  0.4112],
        [ 0.3499,  1.2606],
        [ 0.3096, -1.3506],
        [-0.1291,  0.1212],
        [-0.2857, -0.8869],
        [ 0.1041, -0.4780],
        [ 1.4055, -1.4495],
        [ 1.8769, -0.3176]])

In [48]:
xbow[0]

tensor([[ 0.2811,  0.4112],
        [ 0.3155,  0.8359],
        [ 0.3135,  0.1071],
        [ 0.2029,  0.1106],
        [ 0.1052, -0.0889],
        [ 0.1050, -0.1537],
        [ 0.2908, -0.3388],
        [ 0.4890, -0.3362]])

In [54]:
print(T)

8


In [55]:
# version 2
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1,keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [64]:
# version 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [57]:
xbow[0], xbow2[0]

(tensor([[ 0.2811,  0.4112],
         [ 0.3155,  0.8359],
         [ 0.3135,  0.1071],
         [ 0.2029,  0.1106],
         [ 0.1052, -0.0889],
         [ 0.1050, -0.1537],
         [ 0.2908, -0.3388],
         [ 0.4890, -0.3362]]),
 tensor([[ 0.2811,  0.4112],
         [ 0.3155,  0.8359],
         [ 0.3135,  0.1071],
         [ 0.2029,  0.1106],
         [ 0.1052, -0.0889],
         [ 0.1050, -0.1537],
         [ 0.2908, -0.3388],
         [ 0.4890, -0.3362]]))

In [50]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [52]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [91]:
# version 4: self-attention
torch.manual_seed(1999)
B, T, C  = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C) 

# let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
# out = wei @ v
out = wei @ x

out.shape

torch.Size([4, 8, 32])

In [92]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.7929e-01, 7.2071e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [6.1315e-01, 2.6986e-01, 1.1699e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [6.7289e-02, 2.6367e-01, 8.3458e-02, 5.8558e-01, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.0920e-02, 6.0674e-03, 6.9663e-01, 3.9375e-02, 1.7701e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.4540e-01, 6.0001e-03, 4.6113e-01, 4.7113e-02, 1.1557e-01, 2.2479e-01,
         0.0000e+00, 0.0000e+00],
        [1.5877e-01, 2.3713e-02, 1.7820e-01, 8.8063e-02, 5.7497e-02, 3.2605e-01,
         1.6771e-01, 0.0000e+00],
        [6.6818e-03, 8.7353e-01, 2.6585e-04, 1.9003e-02, 1.6669e-02, 5.9498e-03,
         6.1439e-02, 1.6461e-02]], grad_fn=<SelectBackward0>)

In [99]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [100]:
k.var()

tensor(1.0054)

In [101]:
q.var()

tensor(1.0939)

In [102]:
wei.var()

tensor(1.0153)

In [103]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [104]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])